## Part 1: Preprocessing and EDA

### Problem 1: Dataset Import & Cleaning

In [ ]:
import pandas as pd
%matplotlib inline

In [ ]:
orders = pd.read_csv('data/Orders.csv')

In [ ]:
orders.shape

In [ ]:
pd.options.display.max_columns = 30

In [ ]:
orders.sample(5)

In [ ]:
orders.dtypes

In [ ]:
# convert Profit and Sales columns to numeric type. 
import re
orders.Profit = orders.Profit.map(lambda x: float(re.sub('[$,]', '', x)))
orders.Sales = orders.Sales.map(lambda x: float(re.sub('[,$]', '', x)))

In [ ]:
orders.sample(5)

### Problem 2: Inventory Management
**Is there any seasonal sales trend in the company?**

In [ ]:
# Convert and mutate date columns
orders['Order.Date'] = pd.to_datetime(orders['Order.Date'])
orders['Ship.Date'] = pd.to_datetime(orders['Ship.Date'])
orders['Order.Month'] = orders['Order.Date'].dt.month
orders['Order.Day'] = orders['Order.Date'].dt.dayofyear

In [ ]:
orders.groupby('Order.Day')['Quantity'].sum().plot()

**This plots implies there is seasonal trend, so we can investigate if the series are influenced by the month.**

In [ ]:
orders.groupby('Order.Month')['Quantity'].sum().plot()

**Is the seasonal trend the same for different categories?**

In [ ]:
for index, group in orders.groupby(['Category']):
    group_agg = group.groupby(['Order.Month'])['Quantity'].sum()
    group_agg.plot(y='Quantity', label=index, legend=True)

**Looks like there is a peak during the holiday seasons and big drops in July and October.**

### Problem 3: Why did customers make returns?
**How much profit did we lose due to returns each year?**

In [ ]:
returns = pd.read_csv('data/Returns.csv')

In [ ]:
returns.shape

In [ ]:
returns.sample(5)

In [ ]:
# Region is contained in both dataframes
returns = returns.drop('Region', axis=1).merge(orders, how='left', left_on='Order ID', right_on='Order.ID')

In [ ]:
returns.shape

In [ ]:
returns['Order.Year'] = returns['Order.Date'].dt.year

In [ ]:
returns.dtypes

In [ ]:
# Profit could be negative so we need to sum the absolute values
returns.groupby('Order.Year')['Profit'].apply(lambda c: c.abs().sum()).plot.bar()

**How many customer returned more than once? more than 5 times?**

In [ ]:
len(returns.groupby('Customer.ID').filter(lambda x: len(x) > 1)['Customer.ID'].unique())

In [ ]:
len(returns.groupby('Customer.ID').filter(lambda x: len(x) > 5)['Customer.ID'].unique())

**Which regions are more likely to return orders?**

In [ ]:
returns.groupby('Region').size().sort_values(ascending=False).plot.bar()

**Which categories (sub-categories) of products are more likely to be returned**

In [ ]:
returns.groupby('Category').size().sort_values(ascending=False).plot.bar()

In [ ]:
returns.groupby('Sub.Category').size().sort_values(ascending=False).plot.bar()

## Part II: Machine Learning and Business Use-Case

Now your manager has a basic idea of why customers returned orders. Next, he wants you to use machine learning to predict which orders are most likely to be returned. In this part, you will generate several features based on our previous findings and your manager's requirements.

### Problem 4: Feature Engineering
#### Step 1: Create the dependent variable

In [ ]:
orders['Returned'] = orders['Order.ID'].isin(returns['Order ID'])
orders['Returned'] = orders['Returned'].map(lambda x: 1 if x is True else 0)

In [ ]:
orders.sample(5)

#### Step 2: Time to ship feature

In [51]:
orders['Process.Time'] = orders['Ship.Date']- orders['Order.Date']
orders['Process.Time'] = orders['Process.Time'].dt.days

#### Step 3: Number of return times

In [ ]:
# Need a dataframe to merge later
return_summary = returns.groupby('Product.ID').size().reset_index(name ='Return.Times')

In [ ]:
orders = orders.merge(return_summary, how='left', on='Product.ID', suffixes=['', '_'])
orders['Return.Times'] = orders['Return.Times'].fillna(0)

### Problem 5: Fitting Models

#### Step 1: Select the columns to use

In [ ]:
use_columns = ['Sales', 'Quantity', 'Discount', 'Process.Time', 'Return.Times', 'Shipping.Cost', 'Segment',
               'Ship.Mode', 'Region', 'Category', 'Order.Month', 'Order.Priority','Profit']

In [ ]:
X = pd.get_dummies(orders[use_columns], drop_first=True, dummy_na=True)
y = orders['Returned']

#### Step 2: Train/test split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

#### Step 3: Fit a logistic regression model

In [ ]:
from sklearn.linear_model import LogisticRegression
# Since this is an imbalanced dataset, setting class_weight='balanced' will give us a better result
# http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
LR = LogisticRegression(class_weight='balanced')
LR.fit(X_train, y_train)
y_predict = LR.predict(X_test)

#### Step 4: Evaluate your logistic regression model

In [ ]:
from sklearn.metrics import confusion_matrix, roc_auc_score
confusion_matrix(y_test, y_predict)

In [ ]:
# This is an imbalanced dataset so I would achieve more than 95% accuracy rate even if I always predict 'No'. 
# In this case, accuracy is not a good metric. ROC curve or F1 score works better for this task.
roc_auc_score(y_test, y_predict)

#### Step 5: Fit a random forest model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1000, max_depth=9, class_weight='balanced', n_jobs=-1)
rf.fit(X_train, y_train)
y_predict = rf.predict(X_test)

#### Step 6: Evaluate your random forest model

In [ ]:
confusion_matrix(y_test, y_predict)

In [ ]:
roc_auc_score(y_test, y_predict)

- Now you have multiple models, which one would you pick? 
- Can you get any clue from the confusion matrix? What is the meaning of precision and recall in this case?
- How will your model help the manager make decisions? 
- It really depends your understanding of the business model. For example, if I find a product that has a high probability that will get returned, I would recommend my manager to put it on final sale so the customer get it for a lower price but they are not allowed to return the product. However, if the product turned out to be just fine and we are losing profit because we give it a discount. In that case, I would prefer a model that has a relatively good accuracy with better precision (lower Type I error). 

### Problem 6:
Is there anything wrong with the feature engineering process? How should we fix it?

- The `Return.Times` feature was wrong. If we want to use training data set to fit a model and test dataset to test a model, we assume test dataset is new data we get after fitting the model. But using the whole dataset to count the times will actually indicate these items will be returned in the test dataset. That's why the process is invalid.

- The easiest way to fix the problem is to split the train and test set by different timespan. You can use 2012-2014 data for training and 2015 data as testing. It will mimic the same scenario in real life: **you don't know what's going to happen in the future :)**

- A great [blog post](http://www.fast.ai/2017/11/13/validation-sets/) from fast.ai discussing this problem.